In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.nba_api_draft.draft_history` 
"""
project_id = "perceptive-ivy-290216"
draft = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
draft.head()

,PERSON_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,ROUND_PICK,OVERALL_PICK,DRAFT_TYPE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,ORGANIZATION,ORGANIZATION_TYPE,PLAYER_PROFILE_FLAG
0,1628446,Alpha Kaba,2017,2,30,60,Draft,1610612737,Atlanta,Hawks,ATL,KK Mega Leks (Serbia),Other Team/Club,0
1,1627833,Isaia Cordinier,2016,2,14,44,Draft,1610612737,Atlanta,Hawks,ATL,Denain ASC Voltaire,Other Team/Club,0
2,1626226,Marcus Eriksson,2015,2,20,50,Draft,1610612737,Atlanta,Hawks,ATL,FC Barcelona Basquet (Spain),Other Team/Club,0
3,1626231,Dimitrios Agravanis,2015,2,29,59,Draft,1610612737,Atlanta,Hawks,ATL,Olympiacos B.C. (Greece),Other Team/Club,0
4,201982,Sergiy Gladyr,2009,2,19,49,Draft,1610612737,Atlanta,Hawks,ATL,MBC Mykolaiv (Ukraine),Other Team/Club,0


In [4]:
Season='2023'

In [5]:
draft_season=draft[(draft["SEASON"]==Season)]
draft_season.columns = draft_season.columns.str.replace('_', ' ') 
draft_season.head()

,PERSON ID,PLAYER NAME,SEASON,ROUND NUMBER,ROUND PICK,OVERALL PICK,DRAFT TYPE,TEAM ID,TEAM CITY,TEAM NAME,TEAM ABBREVIATION,ORGANIZATION,ORGANIZATION TYPE,PLAYER PROFILE FLAG
1692,1631340,Mojave King,2023,2,17,47,Draft,1610612747,Los Angeles,Lakers,LAL,Ignite (G League),Other Team/Club,0
3877,1641844,Tarik Biberovic,2023,2,26,56,Draft,1610612763,Memphis,Grizzlies,MEM,Fenerbahce S.K. (Turkey),Other Team/Club,0
4161,1641762,James Nnaji,2023,2,1,31,Draft,1610612765,Detroit,Pistons,DET,FC Barcelona Basquet (Spain),Other Team/Club,0
4600,1641723,Kobe Bufkin,2023,1,15,15,Draft,1610612737,Atlanta,Hawks,ATL,Michigan,College/University,1
4601,1641754,Seth Lundy,2023,2,16,46,Draft,1610612737,Atlanta,Hawks,ATL,Penn State,College/University,1


In [6]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#330072'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    else:
        return '#3860C6'

In [7]:
draft_season['Color'] = draft_season['TEAM ABBREVIATION'].apply(team_color)
draft_season=draft_season.sort_values(by=["OVERALL PICK"])
draft_season=draft_season[['SEASON','PLAYER NAME','OVERALL PICK','TEAM CITY', 'TEAM NAME','ORGANIZATION', 'ORGANIZATION TYPE', 'Color']]

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_8313/1182217914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draft_season['Color'] = draft_season['TEAM ABBREVIATION'].apply(team_color)


In [8]:
import plotly.graph_objects as go

fig = go.Figure(go.Table
 (header={"values": draft_season.columns},
  cells=dict(
    values=draft_season.T.values,
    fill_color=[draft_season.Color],
    align='center', font=dict(color='white', size=12)
    ))
 )

fig.update_layout(
    title="{} NBA Draft Picks".format(Season),
        title_x=0.5,
        # template="plotly_dark",

    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    height=1250,
    width=1200,
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

fig.show()

In [9]:
save_year=2023

In [10]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Draft/{}/{} NBA Draft.html".format(Season,save_year),full_html=False, include_plotlyjs='cdn')